In [1]:
import os
import numpy as np
from scipy.optimize import nnls
from pbwrap.utils import open_image

# Load example image

In [2]:
PATH = "/home/floric/Documents/fish_images/AF_eraser/Data_Carolina/"
images_filenames = os.listdir(PATH)
images_filenames


['scan_006_CPS1_CDKN1A_002_ROI.tif #2.tif',
 'scan_006_dis_ASS1_CTNNB1_002_ROI.tif #3.tif',
 'scan_006_dis_2F2_CCND1_002_ROI.tif #3.tif',
 'scan_007_dis_CPS_CDKN1A_002_ROI.tif #2.tif',
 'scan_007_dis_CPS_CDKN1A_002_ROI.tif #3.tif',
 'scan_006_dis_2F2_CCND1_002_ROI.tif #2.tif',
 'scan_006_ASS1_CTNNB1_002_ROI.tif #2.tif',
 'scan_006_dis_ASS1_CTNNB1_002_ROI.tif #2.tif',
 'scan_006_CPS1_CDKN1A_002_ROI.tif #3.tif',
 'scan_006_ASS1_CTNNB1_002_ROI.tif #3.tif']

In [3]:
signal_filename = 'scan_006_ASS1_CTNNB1_002_ROI.tif #3.tif'
af_filename = 'scan_006_dis_ASS1_CTNNB1_002_ROI.tif #3.tif'

In [4]:
signal_image = open_image(PATH + signal_filename).squeeze()
af_image = open_image(PATH + af_filename).squeeze()

signal_image.shape, af_image.shape

((65, 2048, 2048), (65, 2048, 2048))

In [5]:
images = [signal_image, af_image]

# API testing

In [ ]:
from NMF import remove_autofluorescence_NMF

: 

In [ ]:
signal, af, residuals = remove_autofluorescence_NMF(
    images= images,
    exposure_time = 100,
    gaussian_kernel = 3,
    max_iteration=100
)